# Tuples

 Julia has tuples, very much like Python. They can contain anything:

In [ ]:
t = (1, "Two", 3, 4, 5)

Let's look at one element:

In [ ]:
t[1]

Hey! Did you see that? **Julia is 1-indexed**, like Matlab and other math-oriented programming languages, not 0-indexed like Python and most programming languages. I found it easy to get used to, and in fact I quite like it, but your mileage may vary.

Moreover, the indexing bounds are inclusive. In Python, to get the 1st and 2nd elements of a list or tuple, you would write `t[0:2]` (or just `t[:2]`), while in Julia you write `t[1:2]`.


In [ ]:
t[1:2]

Note that `end` represents the index of the last element in the tuple. So you must write `t[end]` instead of `t[-1]`. Similarly, you must write `t[end - 1]`, not `t[-2]`, and so on.

In [ ]:
t[end]

In [ ]:
t[end - 1:end]

Like in Python, tuples are immutable:

In [ ]:
try
  t[2] = 2
catch ex
  ex
end

The syntax for empty and 1-element tuples is the same as in Python:

In [ ]:
empty_tuple = ()
one_element_tuple = (42,)

You can unpack a tuple, just like in Python (it's called "destructuring" in Julia):

In [ ]:
a, b, c, d, e = (1, "Two", 3, 4, 5)
println("a=$a, b=$b, c=$c, d=$d, e=$e")

It also works with nested tuples, just like in Python:

In [ ]:
(a, (b, c), (d, e)) = (1, ("Two", 3), (4, 5))
println("a=$a, b=$b, c=$c, d=$d, e=$e")

However, consider this example:

In [ ]:
a, b, c = (1, "Two", 3, 4, 5)
println("a=$a, b=$b, c=$c")

In Python, this would cause a `ValueError: too many values to unpack`. In Julia, the extra values in the tuple are just ignored.

If you want to capture the extra values in the variable `c`, you need to do so explicitly:

In [ ]:
t = (1, "Two", 3, 4, 5)
a, b = t[1:2]
c = t[3:end]
println("a=$a, b=$b, c=$c")

Or more concisely:

In [ ]:
(a, b), c = t[1:2], t[3:end]
println("a=$a, b=$b, c=$c")

The corresponding Python code is:

```python
# PYTHON
t = (1, "Two", 3, 4, 5)
a, b, *c = t
print(f"a={a}, b={b}, c={c}")
```

## Named Tuples

Julia supports named tuples:

In [ ]:
nt = (name="Julia", category="Language", stars=5)

In [ ]:
nt.name

In [ ]:
dump(nt)

The corresponding Python code is:

```python
# PYTHON
from collections import namedtuple

Rating = namedtuple("Rating", ["name", "category", "stars"])
nt = Rating(name="Julia", category="Language", stars=5)
print(nt.name) # prints: Julia
print(nt) # prints: Rating(name='Julia', category='Language', stars=5)
```

# Structs
Julia supports structs, which hold multiple named fields, a bit like named tuples:

In [ ]:
struct Person
    name
    age
end

Structs have a default constructor, which expects all the field values, in order:

In [ ]:
p = Person("Mary", 30)

In [ ]:
p.age

You can create other constructors by creating functions with the same name as the struct:

In [ ]:
function Person(name)
    Person(name, -1)
end

function Person()
    Person("no name")
end

p = Person()

This creates two constructors: the second calls the first, which calls the default constructor. Notice that you can create multiple functions with the same name but different arguments. We will discuss this later.

These two constructors are called "outer constructors", since they are defined outside of the definition of the struct. You can also define "inner constructors":

In [ ]:
struct Person2
    name
    age
    function Person2(name)
        new(name, -1)
    end
end

function Person2()
    Person2("no name")
end

p = Person2()

This time, the outer constructor calls the inner constructor, which calls the `new()` function. This `new()` function only works in inner constructors, and of course it creates an instance of the struct.

When you define inner constructors, they replace the default constructor: 

In [ ]:
try
    Person2("Bob", 40)
catch ex
    ex
end

Structs usually have very few inner constructors (often just one), which do the heavy duty work, and the checks. Then they may have multiple outer constructors which are mostly there for convenience.

By default, structs are immutable:

In [ ]:
try
    p.name = "Someone"
catch ex
    ex
end

However, it is possible to define a mutable struct:

In [ ]:
mutable struct Person3
    name
    age
end

p = Person3("Lucy", 79)
p.age += 1
p

Structs look a lot like Python classes, with instance variables and constructors, but where are the methods? We will discuss this later, in the "Methods" section.

# Arrays

Let's create a small array:

In [ ]:
a = [1, 4, 9, 16]

Indexing and assignments work as you would expect:

In [ ]:
a[1] = 10
a[2:3] = [20, 30]
a

## Element Type
Since we used only integers when creating the array, Julia inferred that the array is only meant to hold integers (NumPy arrays behave the same way). Let's try adding a string:

In [ ]:
try
  a[3] = "Three"
catch ex
  ex
end

Nope! We get a `MethodError` exception, telling us that Julia could not convert the string `"Three"` to a 64-bit integer (we will discuss exceptions later). If we want an array that can hold any type, like Python's lists can, we must prefix the array with `Any`, which is Julia's root type (like `object` in Python):

In [ ]:
a = Any[1, 4, 9, 16]
a[3] = "Three"
a

Prefixing with `Float64`, or `String` or any other type works as well:

In [ ]:
Float64[1, 4, 9, 16]

An empty array is automatically an `Any` array:

In [ ]:
a = []

You can use the `eltype()` function to get an array's element type (the equivalent of NumPy arrays' `dtype`):

In [ ]:
eltype([1, 4, 9, 16])

If you create an array containing objects of different types, Julia will do its best to use a type that can hold all the values as precisely as possible. For example, a mix of integers and floats results in a float array:

In [ ]:
[1, 2, 3.0, 4.0]

This is similar to NumPy's behavior:

```python
# PYTHON
np.array([1, 2, 3.0, 4.0]) # => array([1., 2., 3., 4.])
```

A mix of unrelated types results in an `Any` array:

In [ ]:
[1, 2, "Three", 4]

If you want to live in a world without type constraints, you can prefix all you arrays with `Any`, and you will feel like you're coding in Python. But I don't recommend it: the compiler can perform a bunch of optimizations when it knows exactly the type and size of the data the program will handle, so it will run much faster. So when you create an empty array but you know the type of the values it will contain, you might as well prefix it with that type (you don't have to, but it will speed up your program).

## Push and Pop
To append elements to an array, use the `push!()` function. By convention, functions whose name ends with a bang `!` may modify their arguments:

In [ ]:
a = [1]
push!(a, 4)
push!(a, 9, 16)

This is similar to the following Python code:

```python
# PYTHON
a = [1]
a.append(4)
a.extend([9, 16]) # or simply a += [9, 16]
```

And `pop!()` works like in Python:

In [ ]:
pop!(a)

Equivalent to:

```python
# PYTHON
a.pop()
```

There are many more functions you can call on an array. We will see later how to find them.

## Multidimensional Arrays
Importantly, Julia arrays can be multidimensional, just like NumPy arrays:

In [ ]:
M = [1   2   3   4
     5   6   7   8
     9  10  11  12]

Another syntax for this is:

In [ ]:
M = [1 2 3 4; 5 6 7 8; 9 10 11 12]

You can index them much like NumPy arrays:

In [ ]:
M[2:3, 3:4]

You can transpose a matrix using the "adjoint" operator `'`:

In [ ]:
M'

As you can see, Julia arrays are closer to NumPy arrays than to Python lists.


Arrays can be concatenated vertically using the `vcat()` function:

In [ ]:
M1 = [1 2
      3 4]
M2 = [5 6
      7 8]
vcat(M1, M2)

Alternatively, you can use the `[M1; M2]` syntax:

In [ ]:
[M1; M2]

To concatenate arrays horizontally, use `hcat()`:

In [ ]:
hcat(M1, M2)

Or you can use the `[M1 M2]` syntax:

In [ ]:
[M1 M2]

You can combine horizontal and vertical concatenation:

In [ ]:
M3 = [9 10 11 12]
[M1 M2; M3]

Equivalently, you can call the `hvcat()` function. The first argument specifies the number of arguments to concatenate in each block row:

In [ ]:
hvcat((2, 1), M1, M2, M3)

`hvcat()` is useful to create a single cell matrix:

In [ ]:
hvcat(1, 42)

 Or a column vector (i.e., an _n_×1 matrix = a matrix with a single column):

In [ ]:
hvcat((1, 1, 1), 10, 11, 12) # a column vector with values 10, 11, 12
hvcat(1, 10, 11, 12) # equivalent to the previous line

Alternatively, you can transpose a row vector (but `hvcat()` is a bit faster):

In [ ]:
[10 11 12]'

The REPL and IJulia call `display()` to print the result of the last expression in a cell (except when it is `nothing`). It is fairly verbose:

In [ ]:
display([1, 2, 3, 4])

The `println()` function is more concise, but be careful not to confuse vectors, column vectors and row vectors (printed with commas, semi-colons and spaces, respectively):

In [ ]:
println("Vector: ", [1, 2, 3, 4])
println("Column vector: ", hvcat(1, 1, 2, 3, 4))
println("Row vector: ", [1 2 3 4])
println("Matrix: ", [1 2 3; 4 5 6])

Although column vectors are printed as `[1; 2; 3; 4]`, evaluating `[1; 2; 3; 4]` will give you a regular vector. That's because `[x;y]` concatenates `x` and `y` vertically, and if `x` and `y` are scalars or vectors, you just get a regular vector.

|Julia|Python
|-----|------
|`a = [1, 2, 3]` | `a = [1, 2, 3]`<br />or<br />`import numpy as np`<br />`np.array([1, 2, 3])`
|`a[1]` | `a[0]`
|`a[end]` | `a[-1]`
|`a[2:end-1]` | `a[1:-1]`
|`push!(a, 5)` | `a.append(5)`
|`pop!(a)` | `a.pop()`
|`M = [1 2 3]` | `np.array([[1, 2, 3]])`
|`M = [1 2 3]'` | `np.array([[1, 2, 3]]).T`
|`M = hvcat(1,  1, 2, 3)` | `np.array([[1], [2], [3]])`
|`M = [1 2 3`<br />&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;`4 5 6]`<br />or<br />`M = [1 2 3; 4 5 6]` | `M = np.array([[1,2,3], [4,5,6]])`
|`M[1:2, 2:3]` | `M[0:2, 1:3]`
|`[M1; M2]` | `np.r_[M1, M2]`
|`[M1  M2]` | `np.c_[M1, M2]`
|`[M1 M2; M3]` | `np.r_[np.c_[M1, M2], M3]`


## Comprehensions
List comprehensions are available in Julia, just like in Python (they're usually just called "comprehensions" in Julia):

In [ ]:
a = [x^2 for x in 1:4]

You can filter elements using an `if` clause, just like in Python:

In [ ]:
a = [x^2 for x in 1:5 if x ∉ (2, 4)]

* `a ∉ b` is equivalent to `!(a in b)` (or `a not in b` in Python). You can type `∉` with `\notin<tab>`
* `a ∈ b` is equivalent to `a in b`. You can type it with `\in<tab>`

In Julia, comprehensions can contain nested loops, just like in Python:

In [ ]:
a = [(i,j) for i in 1:3 for j in 1:i]

Here's the corresponding Python code:

```python
# PYTHON
a = [(i, j) for i in range(1, 4) for j in range(1, i+1)]
```

Julia comprehensions can also create multi-dimensional arrays (note the different syntax: there is only one `for`):

In [ ]:
a = [row * col for row in 1:3, col in 1:5]

# Dictionaries
The syntax for dictionaries is a bit different than Python:

In [ ]:
d = Dict("tree"=>"arbre", "love"=>"amour", "coffee"=>"café")
println(d["tree"])

In [ ]:
println(get(d, "unknown", "pardon?"))

In [ ]:
keys(d)

In [ ]:
values(d)

In [ ]:
haskey(d, "love")

In [ ]:
"love" in keys(d) # this is slower than haskey()

The equivalent Python code is of course:

```python
d = {"tree": "arbre", "love": "amour", "coffee": "café"}
d["tree"]
d.get("unknown", "pardon?")
d.keys()
d.values()
"love" in d
"love" in d.keys()
```

Dict comprehensions work as you would expect:

In [ ]:
d = Dict(i=>i^2 for i in 1:5)

Note that the items (aka "pairs" in Julia) are shuffled, since dictionaries are hash-based, like in Python (although Python sorts them by key for display).

You can easily iterate through the dictionary's pairs like this:

In [ ]:
for (k, v) in d
    println("$k maps to $v")
end

The equivalent code in Python is:

```python
# PYTHON
d = {i: i**2 for i in range(1, 6)}
for k, v in d.items():
    print(f"{k} maps to {v}")
```

And you can merge dictionaries like this:

In [ ]:
d1 = Dict("tree"=>"arbre", "love"=>"amour", "coffee"=>"café")
d2 = Dict("car"=>"voiture", "love"=>"aimer")

d = merge(d1, d2)

Notice that the second dictionary has priority in case of conflict (it's `"love" => "aimer"`, not `"love" => "amour"`).

In Python, this would be:

```python
# PYTHON
d1 = {"tree": "arbre", "love": "amour", "coffee": "café"}
d2 = {"car": "voiture", "love": "aimer"}
d = {**d1, **d2}
```

Or if you want to update the first dictionary instead of creating a new one:

In [ ]:
merge!(d1, d2)

In Python, that's:

```python
# PYTHON
d1.update(d2)
```

In Julia, each pair is an actual `Pair` object:

In [ ]:
p = "tree" => "arbre"
println(typeof(p))
k, v = p
println("$k maps to $v")

Note that any object for which a `hash()` method is implemented can be used as a key in a dictionary. This includes all the basic types like integers, floats, as well as string, tuples, etc. But it also includes arrays! In Julia, you have the freedom to use arrays as keys (unlike in Python), but make sure not to mutate these arrays after insertion, or else things will break! Indeed, the pairs will be stored in memory in a location that depends on the hash of the key at insertion time, so if that key changes afterwards, you won't be able to find the pair anymore:

In [ ]:
a = [1, 2, 3]
d = Dict(a => "My array")
println("The dictionary is: $d")
println("Indexing works fine as long as the array is unchanged: ", d[a])
a[1] = 10
println("This is the dictionary now: $d")
try
    println("Key changed, indexing is now broken: ", d[a])
catch ex
    ex
end

However, it's still possible to iterate through the keys, the values or the pairs:

In [ ]:
for pair in d
    println(pair)
end

|Julia|Python
|-----|------
|`Dict("tree"=>"arbre", "love"=>"amour")` | `{"tree": "arbre", "love": "amour"}`
|`d["arbre"]` | `d["arbre"]`
|`get(d, "unknown", "default")` | `d.get("unknown", "default")`
|`keys(d)` | `d.keys()`
|`values(d)` | `d.values()`
|`haskey(d, k)` | `k in d`
|`Dict(i=>i^2 for i in 1:4)` | `{i: i**2 for i in 1:4}`
|`for (k, v) in d` | `for k, v in d.items():`
|`merge(d1, d2)` | `{**d1, **d2}`
|`merge!(d1, d2)` | `d1.update(d2)`

# Sets

Let's create a couple sets:

In [ ]:
odd = Set([1, 3, 5, 7, 9, 11])
prime = Set([2, 3, 5, 7, 11])

The order of sets is not guaranteed, just like in Python.

Use `in` or `∈` (type `\in<tab>`) to check whether a set contains a given value:

In [ ]:
5 ∈ odd

In [ ]:
5 in odd

Both of these expressions are equivalent to:

In [ ]:
in(5, odd)

Now let's get the union of these two sets:

In [ ]:
odd ∪ prime

∪ is the union symbol, not a U. To type this character, type `\cup<tab>` (it has the shape of a cup). Alternatively, you can just use the `union()` function:

In [ ]:
union(odd, prime)

Now let's get the intersection using the ∩ symbol (type `\cap<tab>`):

In [ ]:
odd ∩ prime

Or use the `intersect()` function:

In [ ]:
intersect(odd, prime)

Next, let's get the [set difference](https://en.wikipedia.org/wiki/Complement_(set_theory)#Relative_complement) and the [symetric difference](https://en.wikipedia.org/wiki/Symmetric_difference) between these two sets:

In [ ]:
setdiff(odd, prime) # values in odd but not in prime

In [ ]:
symdiff(odd, prime) # values that are not in the intersection

Lastly, set comprehensions work just fine:

In [ ]:
Set([i^2 for i in 1:4])

The equivalent Python code is:

```python
# PYTHON
odds = {1, 3, 5, 7, 9, 11}
primes = {2, 3, 5, 7, 11}
5 in primes
odds | primes # union
odds.union(primes)
odds & primes # intersection
odds.intersection(primes)
odds - primes # set difference
odds.difference(primes)
odds ^ primes # symmetric difference
odds.symmetric_difference(primes)
{i**2 for i in range(1, 5)}
```

Note that you can store any hashable object in a `Set` (i.e., any instance of a type for which the `hash()` method is implemented). This includes arrays, unlike in Python. Just like for dictionary keys, you can add arrays to sets, but make sure not to mutate them after insertion.

|Julia|Python
|-----|------
|`Set([1, 3, 5, 7])` | `{1, 3, 5, 7}`
|`5 in odd` | `5 in odd`
|`Set([i^2 for i in 1:4])` | `{i**2 for i in range(1, 5)}`
|`odd ∪ primes` | `odd | primes`
|`union(odd, primes)` | `odd.union(primes)`
|`odd ∩ primes` | `odd & primes`
|`insersect(odd, primes)` | `odd.intersection(primes)`
|`setdiff(odd, primes)` | `odd - primes` or `odd.difference(primes)`
|`symdiff(odd, primes)` | `odd ^ primes` or `odd.symmetric_difference(primes)`

# Enums

To create an enum, use the `@enum` macro:

In [ ]:
@enum Fruit apple=1 banana=2 orange=3

This creates the `Fruit` enum, with 3 possible values. It also binds the names to the values:

In [ ]:
banana

Or you can get a `Fruit` instance using the value:

In [ ]:
Fruit(2)

And you can get all the instances of the enum easily:

In [ ]:
instances(Fruit)

|Julia|Python
|-----|------
|`@enum Fruit apple=1 banana=2 orange=3` | `from enum import Enum`<br />`class Fruit(Enum):`<br />&nbsp;&nbsp;&nbsp;&nbsp;`APPLE = 1`<br />&nbsp;&nbsp;&nbsp;&nbsp;`BANANA = 2`<br />&nbsp;&nbsp;&nbsp;&nbsp;`ORANGE = 3`
| `Fruit(2) === banana` | `Fruit(2) is Fruit.BANANA`
| `instances(Fruit)` | `dir(Fruit)`


# Object Identity
In the previous example, `Fruit(2)` and `banana` refer to the same object, not just two objects that happen to be equal. You can verify using the `===` operator, which is the equivalent of Python's `is` operator:

In [ ]:
banana === Fruit(2)

You can also check this by looking at their `objectid()`, which is the equivalent of Python's `id()` function:

In [ ]:
objectid(banana)

In [ ]:
objectid(Fruit(2))

In [ ]:
a = [1, 2, 4]
b = [1, 2, 4]
@assert a == b  # a and b are equal
@assert a !== b # but they are not the same object

|Julia|Python
|-----|------
|`a === b` | `a is b`
|`a !== b` | `a is not b`
|`objectid(obj)` | `id(obj)`

# Other Collections

For the Julia equivalent of Python's other collections, namely `defaultdict`, `deque`, `OrderedDict`, and `Counter`, check out these libraries:

* https://github.com/JuliaCollections/DataStructures.jl
* https://github.com/JuliaCollections/OrderedCollections.jl
* https://github.com/andyferris/Dictionaries.jl

Now let's looks at various iteration constructs.

# Iteration Tools

## Generator Expressions

Just like in Python, a generator expression resembles a list comprehension, but without the square brackets, and it returns a generator instead of a list. Here's a much shorter implementation of the `estimate_pi()` function using a generator expression:

In [ ]:
function estimate_pi2(n)
    4 * sum((isodd(i) ? -1 : 1)/(2i+1) for i in 0:n)
end

@assert estimate_pi(100) == estimate_pi2(100)

That's very similar to the corresponding Python code:

```python
# PYTHON
def estimate_pi2(n):
  return 4 * sum((-1 if i%2==1 else 1)/(2*i+1) for i in range(n+1))

assert estimate_pi(100) == estimate_pi2(100)
```

## `zip`, `enumerate`, `collect`

The `zip()` function works much like in Python:

In [ ]:
for (i, s) in zip(10:13, ["Ten", "Eleven", "Twelve"])
    println(i, ": ", s)
end

Notice that the parentheses in `for (i, s)` are required in Julia, as opposed to Python.

The `enumerate()` function also works like in Python, except of course it is 1-indexed:

In [ ]:
for (i, s) in enumerate(["One", "Two", "Three"])
    println(i, ": ", s)
end

To pull the values of a generator into an array, use `collect()`:

In [ ]:
collect(1:5)

A shorter syntax for that is:

In [ ]:
[1:5;]

The equivalent Python code is:

```python
# PYTHON
list(range(1, 6))
```

## Generators
In Python, you can easily write a generator function to create an object that will behave like an iterator. For example, let's create a generator for the Fibonacci sequence (where each number is the sum of the two previous numbers):

```python
def fibonacci(n):
    a, b = 1, 1
    for i in range(n):
      yield a
      a, b = b, a + b

for f in fibonacci(10):
    print(f)
```


This is also quite easy in Julia:

In [ ]:
function fibonacci(n)
    Channel() do ch
        a, b = 1, 1
        for i in 1:n
            put!(ch, a)
            a, b = b, a + b
        end
    end
end

for f in fibonacci(10)
    println(f)
end

The `Channel` type is part of the API for tasks and coroutines. We'll discuss these later.

Now let's take a closer look at functions.